In [12]:
import pandas as pd

In [13]:
teams = ['team1', 'team2', 'team3']
matches = [4, 3, 3]
points = [4, 4, 2]

runsG = [32+28+20+27, 35+38+19, 30+45+26]
ballsB = [24+24+18+22, 24+24+18, 21+24+20]

runsS = [35+30+19+26, 32+45+20, 28+38+27]
ballsF = [24+21+18+20, 24+24+15, 24+24+22]

In [14]:
cols = list(zip(teams, matches, points, runsG, ballsB, runsS, ballsF))
pt = pd.DataFrame(cols, columns=['Team', 'Matches', 'Points', 'Runs Given', 'Balls Bowled', 'Runs Scored', 'Balls Faced'])

In [15]:
pt

,Team,Matches,Points,Runs Given,Balls Bowled,Runs Scored,Balls Faced
0,team1,4,4,107,88,110,83
1,team2,3,4,92,66,97,63
2,team3,3,2,101,65,93,70


In [16]:
def calcNRR(row):
    runs_scored = row['Runs Scored']
    runs_given = row['Runs Given']
    balls_faced = row['Balls Faced']
    balls_bowled = row['Balls Bowled']
    
    nrr = (runs_scored / balls_faced) - (runs_given / balls_bowled)
    return nrr

pt['NRR'] = pt.apply(calcNRR, axis=1)
pt = pt.sort_values(by=['Points', 'NRR'], ascending=[False, False])

In [17]:
pt

,Team,Matches,Points,Runs Given,Balls Bowled,Runs Scored,Balls Faced,NRR
1,team2,3,4,92,66,97,63,0.145743
0,team1,4,4,107,88,110,83,0.109392
2,team3,3,2,101,65,93,70,-0.225275


In [23]:
'''
team2 vs team3
team 2 opt to bat and scored 25 in 3 overs.
updating data into a temp df
'''
tempPT = pt.copy()
tempPT

,Team,Matches,Points,Runs Given,Balls Bowled,Runs Scored,Balls Faced,NRR
1,team2,3,4,92,66,97,63,0.145743
0,team1,4,4,107,88,110,83,0.109392
2,team3,3,2,101,65,93,70,-0.225275


In [24]:
# updating runs scored & balls faced by team2
tempPT.loc[tempPT['Team'] == 'team2', 'Balls Faced'] += 22
tempPT.loc[tempPT['Team'] == 'team2', 'Runs Scored'] += 25

# updating runs given & balls faced by team3
tempPT.loc[tempPT['Team'] == 'team3', 'Balls Bowled'] += 22
tempPT.loc[tempPT['Team'] == 'team3', 'Runs Given'] += 25

tempPT

,Team,Matches,Points,Runs Given,Balls Bowled,Runs Scored,Balls Faced,NRR
1,team2,3,4,92,66,122,85,0.145743
0,team1,4,4,107,88,110,83,0.109392
2,team3,3,2,126,87,93,70,-0.225275


In [25]:
# updating NRR
tempPT['NRR'] = tempPT.apply(calcNRR, axis=1)
tempPT = tempPT.sort_values(by=['Points', 'NRR'], ascending=[False, False])
tempPT

,Team,Matches,Points,Runs Given,Balls Bowled,Runs Scored,Balls Faced,NRR
0,team1,4,4,107,88,110,83,0.109392
1,team2,3,4,92,66,122,85,0.041355
2,team3,3,2,126,87,93,70,-0.119704


In [26]:
tempPT['NRR'].diff()

0         NaN
1   -0.068037
2   -0.161059
Name: NRR, dtype: float64

In [46]:
# counting from max to min
maxBowls = 24
toScore = 26
batting, bowling = 'team3', 'team2'

def check(bef, af):
    bef_index = {team: index for index, team in enumerate(bef)}
    # print(bef_index)
    for i in af:
        # print(bef_index[i], af.index(i))
        if bef_index[i] > af.index(i):
            print(f'{i} overthrown {bef[af.index(i)]} to get to the rank no.{af.index(i)+1}')

for i in range(maxBowls, 0, -1):
    print(f'Scoring in {i} bowls')
    loopDF = tempPT.copy()

    prevRanks = loopDF['Team'].tolist()

    # for batting team
    loopDF.loc[loopDF['Team'] == 'team3', 'Balls Faced'] += i
    loopDF.loc[loopDF['Team'] == 'team3', 'Runs Scored'] += toScore

    # for bowling team
    loopDF.loc[loopDF['Team'] == 'team2', 'Balls Bowled'] += i
    loopDF.loc[loopDF['Team'] == 'team2', 'Runs Given'] += toScore

    # adding a match
    loopDF.loc[loopDF['Team'] == 'team2', 'Matches'] += 1
    loopDF.loc[loopDF['Team'] == 'team3', 'Matches'] += 1

    # updating point
    loopDF.loc[loopDF['Team'] == 'team3', 'Points'] += 2

    loopDF['NRR'] = loopDF.apply(calcNRR, axis=1)
    loopDF = loopDF.sort_values(by=['Points', 'NRR'], ascending=[False, False])
    print(loopDF[['Team', 'Points', 'NRR']])

    print('Bef:', prevRanks)
    afterRanks = loopDF['Team'].tolist()
    print('Af:', afterRanks)

    check(prevRanks, afterRanks)

    print('\n')    

Scoring in 24 bowls
    Team  Points       NRR
1  team2       4  0.124183
0  team1       4  0.109392
2  team3       4 -0.182318
Bef: ['team1', 'team2', 'team3']
Af: ['team2', 'team1', 'team3']
team2 overthrown team1 to get to the rank no.1


Scoring in 23 bowls
    Team  Points       NRR
1  team2       4  0.109451
0  team1       4  0.109392
2  team3       4 -0.168706
Bef: ['team1', 'team2', 'team3']
Af: ['team2', 'team1', 'team3']
team2 overthrown team1 to get to the rank no.1


Scoring in 22 bowls
    Team  Points       NRR
0  team1       4  0.109392
1  team2       4  0.094385
2  team3       4 -0.154798
Bef: ['team1', 'team2', 'team3']
Af: ['team1', 'team2', 'team3']


Scoring in 21 bowls
    Team  Points       NRR
0  team1       4  0.109392
1  team2       4  0.078972
2  team3       4 -0.140584
Bef: ['team1', 'team2', 'team3']
Af: ['team1', 'team2', 'team3']


Scoring in 20 bowls
    Team  Points       NRR
0  team1       4  0.109392
1  team2       4  0.063201
2  team3       4 -0.12605